In [78]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/50startups/50_Startups_dataset.csv


In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from collections import Counter

In [80]:
df = pd.read_csv("/kaggle/input/50startups/50_Startups_dataset.csv")

In [81]:
df.shape

(50, 6)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       50 non-null     int64  
 1   R&D Spend        50 non-null     float64
 2   Administration   50 non-null     float64
 3   Marketing Spend  50 non-null     float64
 4   State            50 non-null     object 
 5   Profit           50 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.5+ KB


In [83]:
df.isna().sum()

Unnamed: 0         0
R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [84]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [85]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle = True)

# Gradient Boosting Regression From Scratch

In [87]:
def get_attribute_type(series):
    if series.dtype == "object":
        return "discrete"
    if str(series.dtype).startswith(("int", "float")):
        return "continuous"
    return "discrete"

In [88]:
class Node:
    def __init__(self, 
                is_leaf = False, 
                label = None, 
                splitting_attribute = None, 
                splitting_attribute_value = None, 
                splitting_attribute_type = None,
                depth = None,
                mse = None):
        self.is_leaf = is_leaf
        self.splitting_attribute = splitting_attribute
        self.splitting_attribute_value = splitting_attribute_value
        self.label = label
        self.children = {}
        self.depth = depth
        self.mse = mse
        self.splitting_attribute_type = splitting_attribute_type

In [89]:
class DecisionTreeRegressor:
    def __init__(self, max_depth = None, min_samples_leaf = 1, min_samples_split = 2, min_impurity_decrease = 0.0, max_features = None):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.max_features = max_features

    def calc_mse(self, y_train):
        y = np.asarray(y_train)
        if len(y) == 0:
            return 0.0
        mean = np.mean(y)
        return np.mean((y - mean)**2)
        
    def calc_gain(self, X_train, y_train, attribute, min_samples_leaf):
        mse = None
        left_over_mse = 0.0
        n = len(y_train)
        if n == 0:
            return (float('-inf'), None)
        info_gain = float('-inf')
        attribute_type = get_attribute_type(X_train[attribute])
        attribute_value = None
        mse = self.calc_mse(y_train)
        if attribute_type == 'discrete':
            values = X_train[attribute].dropna().unique()
            if len(values) <= 1:
                return (float('-inf'), None)
            for av in values:
                mask = X_train[attribute] == av
                labels = y_train[mask]
                if len(labels) < min_samples_leaf:
                    return (float('-inf'), None)
                ni = len(labels)
                left_over_mse += (ni/n) * self.calc_mse(labels)
            info_gain = mse - left_over_mse
        else:
            values = sorted(X_train[attribute].unique())
            min_left_over_mse = mse
            for i in range(len(values)-1):
                av = (values[i] + values[i+1])/2
                left = y_train[X_train[attribute]<=av]
                right = y_train[X_train[attribute]>av]
                if len(left) < min_samples_leaf or len(right) < min_samples_leaf:
                    continue
                    left_over_mse = (len(left)/n) * self.calc_mse(left) + (len(right)/n) * self.calc_mse(right)
                if left_over_mse < min_left_over_mse:
                    min_left_over_mse = left_over_mse
                    attribute_value = av
                    info_gain = mse - left_over_mse
            if attribute_value is None:
                return (float('-inf'), None)
        return (info_gain, attribute_value)
        
    def build_tree(self, X_train, y_train, attributes, depth=0):
        new_node = Node()
        y = np.asarray(y_train)
        value = np.mean(y)
        new_node.label = value
        if (self.max_depth is not None and depth == self.max_depth) or len(y_train) < self.min_samples_split:
            new_node.is_leaf = True
            return new_node
        is_pure = True
        label = y_train.iloc[0]
        new_node.mse = self.calc_mse(y_train)
        new_node.depth = depth
        for curr_label in y_train:
            if curr_label != label:
                is_pure = False
                break
        if is_pure:
            new_node.is_leaf = True
            return new_node
        gains = []
        attributes_subset = attributes.copy()
        if self.max_features == 'sqrt':
            k = int(np.sqrt(len(attributes)))
            attributes_subset = random.sample(attributes, k)
        elif(self.max_features == 'log2'):
            k = int(np.log2(len(attributes)))
            attributes_subset = random.sample(attributes, k)
        elif(type(self.max_features) == int):
            attributes_subset = random.sample(attributes, self.max_features)
        elif(type(self.max_features) == float):
            attributes_subset = random.sample(attributes, int(self.max_features*len(attributes)))
        for attribute in attributes_subset:
            (gain_a, attribute_value) = self.calc_gain(X_train, y_train, attribute, self.min_samples_leaf)
            gains.append((gain_a,(attribute, attribute_value)))
        gains.sort(key=lambda x: x[0], reverse=True)
        best_gain = gains[0][0]
        if best_gain<self.min_impurity_decrease:
            new_node.is_leaf = True
            return new_node
        splitting_attribute = gains[0][1][0]
        splitting_attribute_value = gains[0][1][1]
        new_node.splitting_attribute = splitting_attribute
        new_node.splitting_attribute_value = splitting_attribute_value
        splitting_attribute_type = get_attribute_type(X_train[splitting_attribute])
        new_node.splitting_attribute_type = splitting_attribute_type
        if splitting_attribute_type == 'discrete':   
            for av in X_train[splitting_attribute].unique():
                mask = X_train[splitting_attribute] == av
                new_node.children[av] = self.build_tree(X_train[mask], y_train[mask], attributes,depth = depth +1)
        else :
            mask_left = X_train[splitting_attribute] < splitting_attribute_value
            mask_right = X_train[splitting_attribute] >= splitting_attribute_value
            new_node.children["<="] = self.build_tree(X_train[mask_left], y_train[mask_left], attributes, depth = depth+1)
            new_node.children[">"] = self.build_tree(X_train[mask_right], y_train[mask_right], attributes, depth = depth+1)
        return new_node
        
    def fit(self, X_train, y_train):
        self.root = self.build_tree(X_train, y_train, list(X_train.columns))

    def print_paths(self, root, path):
        if root is None:
            return
        if root.is_leaf:
            for s in path:
                print(s, end = " ")
            print("=> ", root.label)
            return
        for value, child in root.children.items():
            if root.splitting_attribute_type == 'discrete':
                path.append(f"{root.splitting_attribute} (mse = {round(root.mse,3)}) ={value}")
            else:
                path.append(f"{root.splitting_attribute} (mse = {round(root.mse,3)}) {value} {root.splitting_attribute_value}")
            self.print_paths(child, path)
            path.pop()
            
    def print_tree(self):
        self.print_paths(self.root, [])
    def predict(self, X_test):
        y_pred = []
        for i in range(len(X_test)):
            curr = self.root
            while curr and not curr.is_leaf:
                attr = curr.splitting_attribute
                val = X_test.iloc[i][attr]
                if hasattr(val, "item"):
                    val = val.item()
                if curr.splitting_attribute_type == 'discrete':
                    if val in curr.children:
                        curr = curr.children[val]
                    else:
                        break   
                else:
                    if val <= curr.splitting_attribute_value:
                        curr = curr.children["<="]
                    else:
                        curr = curr.children[">"]
            y_pred.append(curr.label)
        return y_pred

    

In [90]:
class GradientBoostingRegressor:
    def __init__(self, learning_rate = 1.0, max_depth = 3, min_samples_leaf = 1, min_samples_split = 2, min_impurity_decrease = 0.0, max_features = None, n_estimators = 100, subsample = 1.0):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.subsample = subsample
        self.estimators = []
        self.initial_prediction = None
        self.learning_rate = learning_rate

    def fit(self, X_train, y_train):
        n = X_train.shape[0]
        self.initial_prediction = np.mean(y_train)
        y_pred = np.full_like(y_train, self.initial_prediction, dtype=float)
        for i in range(self.n_estimators):
            residuals = y_train - y_pred
            if self.subsample < 1.0:
                size = int(self.subsample * n)
                idx = np.random.choice(n, size=size, replace=False)
                X_sub = X_train.iloc[idx].reset_index(drop=True)
                residuals_sub = residuals.iloc[idx].reset_index(drop=True)
            else:
                X_sub = X_train
                residuals_sub = residuals
            clf = DecisionTreeRegressor(max_depth = self.max_depth, min_samples_leaf = self.min_samples_leaf, min_samples_split = self.min_samples_split, max_features = self.max_features, min_impurity_decrease = self.min_impurity_decrease)
            clf.fit(X_sub, residuals_sub)
            update = np.array(clf.predict(X_train))
            y_pred = y_pred + self.learning_rate * update
            self.estimators.append(clf)
    def predict(self, X_test):
        y_pred = np.full(X_test.shape[0], self.initial_prediction)
        for clf in self.estimators:
            y_pred += self.learning_rate * np.array(clf.predict(X_test))
        return y_pred

In [91]:
gb = GradientBoostingRegressor(n_estimators = 200, subsample = 0.4, min_samples_leaf = 3)

In [92]:
gb.fit(X_train, y_train)

In [93]:
y_pred = gb.predict(X_test)

In [94]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.6615882098754442